# 🔍 Chapter 5 练习 3: 敏感性分析 - 因果结论的「压力测试」

## 无混淆假设能验证吗？

在因果推断中，有一个令人不安的事实：

> **无混淆假设 (Unconfoundedness) 无法从数据中验证**

我们永远不知道是否遗漏了某个重要的混淆因子！这就像你永远无法证明「房间里没有隐形的大象」。

那怎么办呢？**敏感性分析 (Sensitivity Analysis)** 来帮忙！

---

## 🎯 学习目标

1. 理解未观测混淆的影响
2. 实现 Rosenbaum 敏感性边界
3. 计算 E-value
4. 进行 Placebo 稳健性检验

## 🌟 敏感性分析的直觉

### 类比：药物试验的「魔鬼代言人」

想象你是 FDA 的审查员，制药公司声称他们的新药有效：

- **他们说**: 「吃药后，患者恢复率高了 20%！」
- **你问**: 「会不会是因为选择吃药的人本来就更健康？」
- **他们说**: 「我们已经控制了所有观测到的因素！」
- **你问**: 「但如果有你们没测量的因素呢？比如患者的乐观程度？」

敏感性分析就是系统地问这个问题：

> **「需要多强的未观测混淆，才能推翻我们的结论？」**

### 两种方法

1. **Rosenbaum 敏感性边界**: 假设存在未观测混淆，看结论何时变得不显著
2. **E-value**: 计算能解释观测关联的最小混淆强度

## 📐 核心公式

### Rosenbaum 敏感性参数 Γ

对于两个协变量相同的个体 i 和 j：

$$\frac{1}{\Gamma} \leq \frac{P(T_i=1|X)}{P(T_j=1|X)} \leq \Gamma$$

- **Γ = 1**: 无未观测混淆（完美的随机化）
- **Γ = 2**: 倾向得分最多相差 2 倍
- **Γ = 3**: 倾向得分最多相差 3 倍

### E-value 公式

$$E = RR + \sqrt{RR \times (RR - 1)}$$

其中 RR 是风险比 (Risk Ratio)。

E-value 表示：**使观测关联完全被混淆解释所需的最小风险比**

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from scipy import stats
from sklearn.linear_model import LinearRegression

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🔍")

## 练习 3.1: 模拟未观测混淆

首先，让我们创建一个有**未观测混淆**的数据集，看看它如何影响因果估计！

### 场景

研究「参加培训」对「薪资」的影响：
- **X**: 观测到的能力指标
- **U**: 未观测的「动机」因素（我们假装看不到）
- **T**: 是否参加培训（受 X 和 U 影响）
- **Y**: 薪资（受 T、X、U 影响）

问题：高动机的人既更可能参加培训，也更可能有高薪资。

In [ ]:
def simulate_unobserved_confounding(
    n: int = 1000,
    confounder_strength: float = 0.5,
    seed: int = 42
) -> Tuple[pd.DataFrame, np.ndarray, Dict]:
    """
    模拟包含未观测混淆的数据
    
    数据生成过程:
    - X ~ N(0, 1)  # 观测协变量（能力）
    - U ~ N(0, 1)  # 未观测混淆因子（动机）
    - P(T=1|X,U) = sigmoid(0.5*X + strength*U)
    - Y = 10 + 2*T + 1.5*X + strength*2*U + noise
    
    真实 ATE = 2.0
    
    Returns:
        (df, U, params)
        df: 包含 X, T, Y 的 DataFrame (不包含 U!)
        U: 未观测混淆因子
        params: 真实参数
    """
    np.random.seed(seed)
    
    # TODO: 生成观测协变量 X (能力)
    X = None  # 你的代码: np.random.randn(n)
    
    # TODO: 生成未观测混淆因子 U (动机)
    U = None  # 你的代码: np.random.randn(n)
    
    # TODO: 生成处理 T (参加培训)
    # 高能力和高动机的人更可能参加培训
    # propensity_logit = 0.5*X + confounder_strength*U
    # propensity = sigmoid(propensity_logit) = 1 / (1 + exp(-logit))
    propensity_logit = None  # 你的代码
    propensity = None  # 你的代码
    T = None  # 你的代码: np.random.binomial(1, propensity)
    
    # TODO: 生成结果 Y (薪资)
    # Y = 10 + 2*T + 1.5*X + confounder_strength*2*U + noise
    # 解读: 培训真正增加薪资 2，能力影响 1.5，动机影响 confounder_strength*2
    noise = np.random.randn(n) * 0.5
    Y = None  # 你的代码
    
    # 创建 DataFrame (假装我们看不到 U!)
    df = pd.DataFrame({
        'X': X,
        'T': T,
        'Y': Y
    })
    
    params = {
        'true_ate': 2.0,
        'confounder_strength': confounder_strength
    }
    
    return df, U, params

In [ ]:
# 计算不同的 ATE 估计
def compute_naive_ate(df: pd.DataFrame) -> float:
    """
    朴素 ATE 估计: E[Y|T=1] - E[Y|T=0]
    (忽略所有混淆)
    """
    # TODO: 计算朴素估计
    # 你的代码
    pass


def compute_adjusted_ate(df: pd.DataFrame) -> float:
    """
    调整 X 后的 ATE 估计
    使用线性回归: Y ~ T + X
    """
    # TODO: 使用回归估计 ATE
    # 1. 构建特征矩阵 [[T1, X1], [T2, X2], ...]
    # 2. 拟合线性回归
    # 3. 返回 T 的系数
    
    # 你的代码
    pass

In [ ]:
# 测试不同混淆强度
print("=== 未观测混淆的影响 ===")
print(f"真实 ATE = 2.0\n")

strengths = [0.0, 0.3, 0.6, 1.0]
results = []

for strength in strengths:
    df, U, params = simulate_unobserved_confounding(
        n=2000,
        confounder_strength=strength
    )
    
    if df is not None and df['X'].iloc[0] is not None:
        naive_ate = compute_naive_ate(df)
        adjusted_ate = compute_adjusted_ate(df)
        
        if naive_ate is not None and adjusted_ate is not None:
            results.append({
                'Strength': strength,
                'Naive ATE': naive_ate,
                'Naive Bias': naive_ate - 2.0,
                'Adjusted ATE': adjusted_ate,
                'Adjusted Bias': adjusted_ate - 2.0
            })

if len(results) > 0:
    results_df = pd.DataFrame(results)
    display(results_df.round(4))
    
    print("\n解读:")
    print("- 混淆强度 = 0: 无混淆，两种估计都接近真实值")
    print("- 混淆强度增加: 朴素估计偏差增大（高估效应）")
    print("- 调整 X 后: 仍有偏差，因为 U 未被控制！")
else:
    print("[TODO] 请完成数据生成和估计函数")

In [ ]:
# 可视化混淆的影响
if len(results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 估计值 vs 混淆强度
    axes[0].plot(results_df['Strength'], results_df['Naive ATE'], 
                'bo-', label='朴素估计', linewidth=2, markersize=8)
    axes[0].plot(results_df['Strength'], results_df['Adjusted ATE'], 
                'gs-', label='调整 X 后', linewidth=2, markersize=8)
    axes[0].axhline(y=2.0, color='red', linestyle='--', label='真实 ATE')
    axes[0].set_xlabel('未观测混淆强度')
    axes[0].set_ylabel('ATE 估计')
    axes[0].set_title('估计值随混淆强度变化')
    axes[0].legend()
    
    # 偏差 vs 混淆强度
    axes[1].bar(np.arange(len(strengths)) - 0.2, results_df['Naive Bias'], 
               0.35, label='朴素估计偏差', color='#3498db')
    axes[1].bar(np.arange(len(strengths)) + 0.2, results_df['Adjusted Bias'], 
               0.35, label='调整后偏差', color='#2ecc71')
    axes[1].axhline(y=0, color='red', linestyle='--')
    axes[1].set_xticks(np.arange(len(strengths)))
    axes[1].set_xticklabels([f'{s:.1f}' for s in strengths])
    axes[1].set_xlabel('未观测混淆强度')
    axes[1].set_ylabel('偏差 (估计 - 真实)')
    axes[1].set_title('偏差随混淆强度变化')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 练习 3.2: Rosenbaum 敏感性边界

现在让我们实现 Rosenbaum 敏感性分析！

核心问题：**如果存在未观测混淆，我们的结论会有多脆弱？**

In [ ]:
def compute_rosenbaum_bounds(
    Y: np.ndarray,
    T: np.ndarray,
    gamma: float
) -> Tuple[float, float]:
    """
    计算 Rosenbaum 敏感性边界
    
    Γ 参数的含义:
    - Γ = 1: 假设无未观测混淆
    - Γ > 1: 允许倾向得分最多相差 Γ 倍
    
    Args:
        Y: 结果变量
        T: 处理变量
        gamma: 敏感性参数
    
    Returns:
        (lower_bound, upper_bound)
    """
    # 观测的 ATE
    ate_obs = Y[T == 1].mean() - Y[T == 0].mean()
    
    if gamma == 1.0:
        # 无混淆假设下，边界就是点估计
        return ate_obs, ate_obs
    
    # TODO: 计算敏感性边界
    # 简化版本: 边界宽度与 gamma 和标准误相关
    
    # 1. 计算 ATE 的标准误差
    n1 = (T == 1).sum()
    n0 = (T == 0).sum()
    var1 = Y[T == 1].var()
    var0 = Y[T == 0].var()
    
    se = None  # 你的代码: np.sqrt(var1/n1 + var0/n0)
    
    # 2. 边界宽度随 gamma 增加
    # 简化公式: bound_width = se * log(gamma) * 2
    bound_width = None  # 你的代码
    
    # 3. 计算上下界
    lower_bound = None  # 你的代码: ate_obs - bound_width
    upper_bound = None  # 你的代码: ate_obs + bound_width
    
    return lower_bound, upper_bound

In [ ]:
def sensitivity_curve(
    Y: np.ndarray,
    T: np.ndarray,
    gamma_range: np.ndarray,
    true_ate: float = None
) -> pd.DataFrame:
    """
    计算敏感性曲线
    """
    results = []
    
    for gamma in gamma_range:
        lower, upper = compute_rosenbaum_bounds(Y, T, gamma)
        
        row = {
            'gamma': gamma,
            'lower': lower,
            'upper': upper
        }
        
        if true_ate is not None:
            row['true_ate'] = true_ate
        
        results.append(row)
    
    return pd.DataFrame(results)

In [ ]:
# 生成有混淆的数据
df, U, params = simulate_unobserved_confounding(
    n=2000,
    confounder_strength=0.6
)

if df is not None and df['Y'].iloc[0] is not None:
    # 计算敏感性曲线
    gamma_range = np.linspace(1.0, 5.0, 30)
    
    try:
        sens_df = sensitivity_curve(
            df['Y'].values,
            df['T'].values,
            gamma_range,
            true_ate=params['true_ate']
        )
        
        if sens_df is not None and sens_df['lower'].iloc[0] is not None:
            print("敏感性分析结果:")
            print("="*60)
            print(f"\n观测 ATE: {df['Y'][df['T']==1].mean() - df['Y'][df['T']==0].mean():.4f}")
            print(f"真实 ATE: {params['true_ate']:.4f}")
            
            print("\n部分结果:")
            display(sens_df.iloc[::5].round(4))  # 每5行显示一次
            
            # 找到包含 0 的最小 gamma
            includes_zero = (sens_df['lower'] <= 0) & (sens_df['upper'] >= 0)
            if includes_zero.any():
                gamma_threshold = sens_df.loc[includes_zero, 'gamma'].min()
                print(f"\n敏感性阈值: Γ = {gamma_threshold:.2f}")
                print(f"解读: 如果未观测混淆使倾向得分相差 {gamma_threshold:.1f} 倍，")
                print(f"      效应可能变为零（不显著）")
            else:
                print(f"\n在 Γ ≤ {gamma_range.max():.1f} 范围内，效应始终显著")
                print("结论相对稳健！")
        else:
            print("[TODO] 请完成 compute_rosenbaum_bounds 函数")
    except Exception as e:
        print(f"[错误] {e}")
else:
    print("[TODO] 请完成数据生成")

In [ ]:
# 可视化敏感性分析
if 'sens_df' in dir() and sens_df is not None and sens_df['lower'].iloc[0] is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # 绘制边界
    ax.fill_between(
        sens_df['gamma'], 
        sens_df['lower'], 
        sens_df['upper'],
        alpha=0.3, color='#3498db', label='敏感性区间'
    )
    ax.plot(sens_df['gamma'], sens_df['lower'], 'b-', linewidth=2, label='下界')
    ax.plot(sens_df['gamma'], sens_df['upper'], 'b-', linewidth=2, label='上界')
    
    # 真实 ATE
    ax.axhline(y=params['true_ate'], color='green', linestyle='--', 
              linewidth=2, label=f'真实 ATE = {params["true_ate"]}')
    
    # 零线
    ax.axhline(y=0, color='red', linestyle='-', linewidth=1, label='零效应')
    
    # 找到交叉点
    includes_zero = (sens_df['lower'] <= 0) & (sens_df['upper'] >= 0)
    if includes_zero.any():
        gamma_threshold = sens_df.loc[includes_zero, 'gamma'].min()
        ax.axvline(x=gamma_threshold, color='orange', linestyle=':', 
                  linewidth=2, label=f'敏感性阈值 Γ={gamma_threshold:.2f}')
    
    ax.set_xlabel('Γ (敏感性参数)', fontsize=12)
    ax.set_ylabel('ATE 估计', fontsize=12)
    ax.set_title('Rosenbaum 敏感性分析\n"结论对未观测混淆有多脆弱?"', fontsize=14)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n解读指南:")
    print("- Γ = 1: 假设无未观测混淆")
    print("- 随着 Γ 增加，允许更强的未观测混淆")
    print("- 当区间包含 0 时，结论变得不确定")
    print("- 敏感性阈值越高，结论越稳健")

## 练习 3.3: E-value

E-value 是另一种流行的敏感性度量，由 VanderWeele & Ding (2017) 提出。

**核心问题**: 需要多强的未观测混淆，才能完全解释观测到的关联？

In [ ]:
def ate_to_risk_ratio(ate: float, baseline_mean: float) -> float:
    """
    将 ATE 转换为风险比 (Risk Ratio)
    
    RR = (baseline + ATE) / baseline
    
    TODO: 计算风险比
    """
    # 你的代码
    pass


def compute_e_value(observed_rr: float, ci_lower: float = None) -> Dict[str, float]:
    """
    计算 E-value
    
    E-value: 使观测关联完全被混淆解释所需的最小风险比
    
    公式: E = RR + sqrt(RR * (RR - 1))  [当 RR >= 1]
          E = 1/RR + sqrt(1/RR * (1/RR - 1))  [当 RR < 1]
    
    Args:
        observed_rr: 观测到的风险比
        ci_lower: 置信区间下界 (可选)
    
    Returns:
        {'e_value': ..., 'e_value_ci': ...}
    """
    # 确保 RR >= 1 (如果 < 1，取倒数)
    if observed_rr < 1:
        observed_rr = 1 / observed_rr
    
    # TODO: 计算 E-value
    # E = RR + sqrt(RR * (RR - 1))
    e_value = None  # 你的代码
    
    result = {'e_value': e_value}
    
    # 如果有置信区间下界，也计算其 E-value
    if ci_lower is not None and ci_lower > 1:
        e_value_ci = ci_lower + np.sqrt(ci_lower * (ci_lower - 1))
        result['e_value_ci'] = e_value_ci
    
    return result

In [ ]:
# 测试 E-value
if df is not None and df['Y'].iloc[0] is not None:
    try:
        # 计算观测 ATE
        naive_ate = df['Y'][df['T']==1].mean() - df['Y'][df['T']==0].mean()
        baseline_mean = df.loc[df['T'] == 0, 'Y'].mean()
        
        # 转换为风险比
        rr = ate_to_risk_ratio(naive_ate, baseline_mean)
        
        if rr is not None:
            # 计算 E-value
            e_values = compute_e_value(rr)
            
            if e_values['e_value'] is not None:
                print("E-value 分析:")
                print("="*50)
                print(f"\n观测 ATE: {naive_ate:.4f}")
                print(f"基线均值: {baseline_mean:.4f}")
                print(f"风险比 (RR): {rr:.4f}")
                print(f"\nE-value: {e_values['e_value']:.4f}")
                
                print(f"\n解读:")
                print(f"  需要一个与处理和结果都有 RR ≥ {e_values['e_value']:.2f} 关联的")
                print(f"  未观测混淆因子，才能完全解释观测到的效应")
                
                if e_values['e_value'] > 3:
                    print(f"\n  评价: E-value 较高，结论相对稳健")
                else:
                    print(f"\n  评价: E-value 较低，结论可能对混淆敏感")
            else:
                print("[TODO] 请完成 compute_e_value 函数")
        else:
            print("[TODO] 请完成 ate_to_risk_ratio 函数")
    except Exception as e:
        print(f"[错误] {e}")

## 练习 3.4: Placebo 稳健性检验

Placebo 测试是一种简单但强大的稳健性检验：

**思路**: 如果我们的因果估计方法是正确的，那么对于「不应受处理影响的变量」，我们不应该检测到显著效应。

例如：
- 研究「培训」对「薪资」的影响
- Placebo 变量：参加培训前的身高（显然培训不应该改变身高）
- 如果我们在身高上也发现了「效应」，说明可能有问题！

In [ ]:
def placebo_test(
    df: pd.DataFrame,
    outcome_col: str = 'Y',
    placebo_outcome_col: str = 'Y_placebo'
) -> Dict[str, float]:
    """
    Placebo 测试
    
    使用不应受处理影响的"假"结果变量进行测试
    
    Args:
        df: 包含 T, Y, Y_placebo 的 DataFrame
        outcome_col: 真实结果列名
        placebo_outcome_col: 假结果列名
    
    Returns:
        {'true_effect': ..., 'placebo_effect': ..., 'p_value_placebo': ...}
    """
    T = df['T'].values
    
    # TODO: 计算真实结果的效应
    Y = df[outcome_col].values
    true_effect = None  # 你的代码: Y[T==1].mean() - Y[T==0].mean()
    
    # TODO: 计算假结果的效应
    Y_placebo = df[placebo_outcome_col].values
    placebo_effect = None  # 你的代码
    
    # TODO: t 检验测试假结果效应是否显著
    # 使用 scipy.stats.ttest_ind
    t_stat, p_value = None, None  # 你的代码
    
    return {
        'true_effect': true_effect,
        'placebo_effect': placebo_effect,
        'p_value_placebo': p_value
    }

In [ ]:
# 测试 Placebo 检验
if df is not None and df['Y'].iloc[0] is not None:
    # 生成假结果变量（不应受 T 影响，只与 X 相关）
    np.random.seed(123)
    df['Y_placebo'] = df['X'] * 2 + np.random.randn(len(df)) * 0.5
    
    try:
        placebo_results = placebo_test(df, 'Y', 'Y_placebo')
        
        if placebo_results['true_effect'] is not None:
            print("Placebo 测试结果:")
            print("="*50)
            print(f"\n真实结果 (Y) 的效应: {placebo_results['true_effect']:.4f}")
            print(f"假结果 (Y_placebo) 的效应: {placebo_results['placebo_effect']:.4f}")
            
            if placebo_results['p_value_placebo'] is not None:
                print(f"假结果 p-value: {placebo_results['p_value_placebo']:.4f}")
                
                print("\n解读:")
                if placebo_results['p_value_placebo'] < 0.05:
                    print("  ⚠️ 警告: 假结果也显示显著效应!")
                    print("  这可能意味着:")
                    print("  - 存在未控制的混淆")
                    print("  - 选择偏差问题")
                    print("  - 模型设定错误")
                else:
                    print("  ✅ 假结果无显著效应，通过 Placebo 测试!")
                    print("  这增强了我们对因果估计的信心")
        else:
            print("[TODO] 请完成 placebo_test 函数")
    except Exception as e:
        print(f"[错误] {e}")

In [ ]:
# 对比有混淆和无混淆情况下的 Placebo 测试
print("对比不同混淆强度下的 Placebo 测试:")
print("="*60)

placebo_comparison = []

for strength in [0.0, 0.3, 0.6, 1.0]:
    df_temp, U_temp, params_temp = simulate_unobserved_confounding(
        n=2000,
        confounder_strength=strength
    )
    
    if df_temp is not None and df_temp['Y'].iloc[0] is not None:
        # 生成依赖于 U 的假结果（如果 U 是混淆，这个也会被"影响"）
        df_temp['Y_placebo'] = U_temp * 2 + np.random.randn(len(df_temp)) * 0.5
        
        results = placebo_test(df_temp, 'Y', 'Y_placebo')
        
        if results['placebo_effect'] is not None:
            placebo_comparison.append({
                'Strength': strength,
                'True Effect': results['true_effect'],
                'Placebo Effect': results['placebo_effect'],
                'P-value': results['p_value_placebo']
            })

if len(placebo_comparison) > 0:
    comp_df = pd.DataFrame(placebo_comparison)
    display(comp_df.round(4))
    
    print("\n解读:")
    print("- 当混淆强度为 0 时，Placebo 效应接近 0")
    print("- 随着混淆强度增加，Placebo 效应变得显著")
    print("- 这说明 Placebo 测试可以帮助检测混淆!")

## 🤔 思考题

In [ ]:
# 思考题 1: 为什么无混淆假设无法从数据中验证？

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: Rosenbaum Γ = 2 意味着什么？在实践中这算强混淆还是弱混淆？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: E-value 和 Rosenbaum bounds 有何异同？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 在什么情况下应该报告敏感性分析结果？

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: 如果敏感性分析显示结论对未观测混淆很敏感，应该怎么办？

answer_5 = """
你的答案:



"""
print(answer_5)

In [ ]:
# 思考题 6: Placebo 测试的原理是什么？它能检测哪种类型的偏差？

answer_6 = """
你的答案:



"""
print(answer_6)

## 📊 总结

### 敏感性分析方法对比

| 方法 | 回答的问题 | 优点 | 缺点 |
|------|------------|------|------|
| Rosenbaum Bounds | 多强的混淆会推翻结论？ | 直观、有理论支撑 | 需要匹配 |
| E-value | 解释效应需要多强的混淆？ | 简单计算、广泛适用 | 假设较强 |
| Placebo Test | 方法是否可靠？ | 直接检验假设 | 需要好的 placebo 变量 |

### 何时使用敏感性分析？

✅ **总是应该做**:
- 观察性研究
- 无法随机化的情况
- 结论有重要政策含义

### 如何解读结果？

| E-value / Γ 阈值 | 解读 |
|------------------|------|
| < 1.5 | 结论非常脆弱 |
| 1.5 - 2.5 | 中等稳健 |
| 2.5 - 4.0 | 较为稳健 |
| > 4.0 | 非常稳健 |

### 最佳实践

1. **总是报告敏感性分析**
2. **结合领域知识**判断假设的混淆是否合理
3. **使用多种方法**交叉验证
4. **诚实地讨论局限性**

In [ ]:
print("🔍 恭喜完成敏感性分析练习!")
print("\n你已经学会了:")
print("  ✓ 理解未观测混淆的影响")
print("  ✓ Rosenbaum 敏感性边界")
print("  ✓ E-value 计算和解读")
print("  ✓ Placebo 稳健性检验")
print("\n恭喜完成 Chapter 5 所有练习!")
print("下一步: Chapter 6 应用场景 - 优惠券优化!")